# Obtén datos de la API de Marvel

![imagen](https://cdn.hobbyconsolas.com/sites/navi.axelspringer.es/public/styles/hc_1440x810/public/media/image/2021/09/marvel-2460339.jpg?itok=cBnC1CBi)

Te tendrás que [registrar](https://developer.marvel.com/) y consultar la [documentación](https://developer.marvel.com/docs)

Queremos que consultes a la api para que te devuelva la info de los personajes de marvel que empiecen por la inicial de tu nombre. Deberemos guardarlos en un csv la información con la siguiente estructura

![imagen](./img/Captura_marvel.PNG)

Aquí te dejamos el código en python para poder empezar a hacer las llamadas, rellenando algunas variables como tus keys, parámetros de la llamada y la url (endpoint) a la que quieres acceder

In [19]:
import hashlib
import requests
import datetime
import pandas as pd

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

In [20]:
def hash_params(timestamp,priv_key,pub_key):
    """ Marvel API requires server side API calls to include
    md5 hash of timestamp + public key + private key """

    hash_md5 = hashlib.md5()
    hash_md5.update(f'{timestamp}{priv_key}{pub_key}'.encode('utf-8'))
    hashed_params = hash_md5.hexdigest()

    return hashed_params

In [35]:
timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')

pub_key = '7b519eca220cea84b556d6da07c75bbe'
priv_key = 'd097b014e0a40e56706de63cd61b3a954601a1de'

params = {'ts': timestamp, 
        'apikey': pub_key, 
        'hash': hash_params(timestamp,priv_key,pub_key),
        'nameStartsWith':"M",   # Cambia este valor para la inicial de tu nombre
        'offset':100,         # Juega con este valor
        'limit': 100            # Juega con este valor
        }

url = 'http://gateway.marvel.com/v1/public/characters'

res = requests.get(url,params=params)

In [34]:
import requests
import datetime
import hashlib

def hash_params(timestamp, priv_key, pub_key):
    hash_input = f"{timestamp}{priv_key}{pub_key}"
    return hashlib.md5(hash_input.encode('utf-8')).hexdigest()

timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')
pub_key = '7b519eca220cea84b556d6da07c75bbe'
priv_key = 'd097b014e0a40e56706de63cd61b3a954601a1de'

url = 'http://gateway.marvel.com/v1/public/characters'


all_results = []
offset = 0
limit = 100

while True:
    params = {
        'ts': timestamp,
        'apikey': pub_key,
        'hash': hash_params(timestamp, priv_key, pub_key),
        'nameStartsWith': "M",
        'offset': offset,
        'limit': limit
    }

    print(f"Solicitando con offset: {offset}")

    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        results = data['data']['results']
        all_results.extend(results)
        
        print(f"Obtenidos {len(results)} resultados. Total acumulado: {len(all_results)}")
        
        if len(results) == 0:  # Si no hay resultados, hemos llegado al final
            break
        
        offset += 1  # Incrementamos el offset en 100 para la siguiente iteración
    else:
        print(f"Error en la solicitud: {response.status_code}")
        break

print(f"Total de resultados obtenidos: {len(all_results)}")

Solicitando con offset: 0
Obtenidos 100 resultados. Total acumulado: 100
Solicitando con offset: 10
Obtenidos 100 resultados. Total acumulado: 200
Solicitando con offset: 20
Obtenidos 100 resultados. Total acumulado: 300
Solicitando con offset: 30
Obtenidos 100 resultados. Total acumulado: 400
Solicitando con offset: 40
Obtenidos 100 resultados. Total acumulado: 500
Solicitando con offset: 50
Obtenidos 100 resultados. Total acumulado: 600
Solicitando con offset: 60
Obtenidos 97 resultados. Total acumulado: 697
Solicitando con offset: 70
Obtenidos 87 resultados. Total acumulado: 784
Solicitando con offset: 80
Obtenidos 77 resultados. Total acumulado: 861
Solicitando con offset: 90
Obtenidos 67 resultados. Total acumulado: 928
Solicitando con offset: 100
Obtenidos 57 resultados. Total acumulado: 985
Solicitando con offset: 110
Obtenidos 47 resultados. Total acumulado: 1032
Solicitando con offset: 120
Obtenidos 37 resultados. Total acumulado: 1069
Solicitando con offset: 130
Obtenidos 27 

In [36]:
res

<Response [200]>

In [17]:
response

<Response [200]>

In [39]:
respuesta_json = res.json()
respuesta_json

{'code': 200,
 'status': 'Ok',
 'copyright': '© 2024 MARVEL',
 'attributionText': 'Data provided by Marvel. © 2024 MARVEL',
 'attributionHTML': '<a href="http://marvel.com">Data provided by Marvel. © 2024 MARVEL</a>',
 'etag': 'b5683285336f94b3feb8d7e998a7b9fbb386b99e',
 'data': {'offset': 100,
  'limit': 100,
  'total': 157,
  'count': 57,
  'results': [{'id': 1009446,
    'name': 'Mister Fear',
    'description': '',
    'modified': '1969-12-31T19:00:00-0500',
    'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available',
     'extension': 'jpg'},
    'resourceURI': 'http://gateway.marvel.com/v1/public/characters/1009446',
    'comics': {'available': 4,
     'collectionURI': 'http://gateway.marvel.com/v1/public/characters/1009446/comics',
     'items': [{'resourceURI': 'http://gateway.marvel.com/v1/public/comics/20635',
       'name': 'Daredevil (1998) #105'},
      {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/8377',
       'name': 'Darede

In [18]:
respuesta_json_v2 = respuesta_json()
respuesta_json_v2

TypeError: 'dict' object is not callable

In [38]:
import pandas as pd

# en respuesta_json
characters = respuesta_json['data']['results']

# Crear una lista de diccionarios con la información requerida
data = []
for character in characters:
    data.append({
        'id': character['id'],
        'name': character['name'],
        'picture_url': f"{character['thumbnail']['path']}.{character['thumbnail']['extension']}"
    })

# Crear el DataFrame
df = pd.DataFrame(data)

# Mostrar el DataFrame
df

,id,name,picture_url
0,1009446,Mister Fear,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
1,1009447,Mister Sinister,http://i.annihil.us/u/prod/marvel/i/mg/5/80/526034ac3c53a.jpg
2,1017318,Mister Sinister (Deadpool),http://i.annihil.us/u/prod/marvel/i/mg/e/10/5239c24883863.jpg
3,1010948,Mister Sinister (House of M),http://i.annihil.us/u/prod/marvel/i/mg/f/00/4c00340427a12.jpg
4,1011254,Mister Sinister (Ultimate),http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
5,1010682,Misty Knight,http://i.annihil.us/u/prod/marvel/i/mg/d/90/511151789a863.jpg
6,1011220,Mockingbird,http://i.annihil.us/u/prod/marvel/i/mg/9/b0/51e829af23af9.jpg
7,1009410,Moira MacTaggert,http://i.annihil.us/u/prod/marvel/i/mg/5/d0/4c003d5e64f22.jpg
8,1010941,Moira MacTaggert (Ultimate),http://i.annihil.us/u/prod/marvel/i/mg/8/d0/4c7c640312deb.jpg
9,1009448,Mojo,http://i.annihil.us/u/prod/marvel/i/mg/8/a0/4c003d3444dce.jpg


In [31]:
df.to_csv("data/marvelsuperheroestrue.csv", sep = ';', index = True)